In [1]:
import json
import time
import random
import pymongo
import copy
import MyCommon
from pymongo import MongoClient
from collections import deque
import cassiopeia
from cassiopeia import riotapi
import numpy as np
from sklearn import preprocessing, cluster, decomposition
from scipy.cluster.vq import kmeans,vq
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans, AffinityPropagation
from sklearn.decomposition import FastICA, PCA
from sklearn import metrics

riotapi.set_region("KR")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.set_rate_limits((10, 10), (500, 600))
riotapi.print_calls(True)

client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.summoners
matches_collection = loladb.matches
print summoners_collection.count()
print matches_collection.count()

1255
18749


In [2]:
champions = riotapi.get_champions()
dict_id2idx = dict()
dict_idx2id = dict()
dict_id2key = dict()
for a_idx, a_champion in enumerate(champions):
    champ_id = a_champion.id
    champ_key = a_champion.key
    dict_id2idx[champ_id] = a_idx
    dict_idx2id[a_idx] = champ_id
    dict_id2key[champ_id] = champ_key
    
#print dict_id2key[dict_idx2id[132]]
#print dict_id2key

def GetIdxFromId(a_id):
    return dict_id2idx[a_id]

def GetIdFromIdx(a_idx):
    return dict_idx2id[a_idx]

def GetNameFromId(a_id):
    return dict_id2key[a_id]

https://global.api.pvp.net/api/lol/static-data/kr/v1.2/champion?champData=all&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://global.api.pvp.net/api/lol/static-data/kr/v1.2/item?itemListData=all&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24


In [3]:
#count the histogram of jungle champions
#print matches_collection.find({'matchVersion':'6.24.168.1268'}).count()
CHAMP_NUM = 133
jungle_champions = np.zeros((CHAMP_NUM))
for a_idx, a_match in enumerate(matches_collection.find({'matchVersion':'6.24.168.1268'})):
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue #do not count draw game, if any

    a_duration = a_match['matchDuration']
    a_minute = a_duration/60
    if a_minute < 20:
        continue #do not count remake game
        
    participants = a_match['participants']
    for a_participant in participants[0:5]:
        if a_participant['timeline']['lane'] == 'JUNGLE':
            champ_id = a_participant['championId']
            if champ_id == 164:
                break# break for new champion
            champ_idx = GetIdxFromId(champ_id)
            jungle_champions[champ_idx] += 1
            break
    for a_participant in participants[5:10]:
        if a_participant['timeline']['lane'] == 'JUNGLE':
            champ_id = a_participant['championId']
            if champ_id == 164:
                break# break for new champion
            champ_idx = GetIdxFromId(champ_id)
            jungle_champions[champ_idx] += 1
            break

In [5]:
sorted_idx = np.argsort(jungle_champions)
total_game = sum(jungle_champions)/2

top_jungles = []
TOP_NUM = 20
print 'Number,Percentage,Name'
for i in range(TOP_NUM):
    print str(jungle_champions[sorted_idx[132 - TOP_NUM + 1 + i]]) 

for i in range(TOP_NUM):
    print str(jungle_champions[sorted_idx[132 - TOP_NUM + 1 + i]]/total_game)

for i in range(TOP_NUM):
    print GetNameFromId(GetIdFromIdx(sorted_idx[132 - TOP_NUM + 1 + i]))
    top_jungles.append(GetIdFromIdx(sorted_idx[132 - TOP_NUM + 1 + i]))

Number,Percentage,Name
109.0
116.0
126.0
137.0
164.0
168.0
229.0
265.0
290.0
307.0
319.0
594.0
778.0
815.0
898.0
1849.0
2155.0
2450.0
2861.0
5038.0
0.0102511050503
0.0109094328976
0.0118499012508
0.0128844164394
0.0154236809931
0.0157998683344
0.0215367252892
0.0249224113609
0.027273582244
0.0288723784445
0.0300009404684
0.0558638201825
0.0731684378821
0.0766481707891
0.0844540581209
0.173892598514
0.202670930123
0.230414746544
0.269067995862
0.473807956362
MasterYi
Rammus
JarvanIV
Gragas
Kindred
Zac
Vi
Shaco
Olaf
Ivern
Shyvana
Nidalee
Rengar
Hecarim
Evelynn
Khazix
Graves
RekSai
Elise
LeeSin


In [6]:
#winrate
#for a_jg_id in top_jungles:
    #print a_jg_id
    #print GetNameFromId(a_jg_id)
top_jungle_idx = {}
top_sum = np.zeros(TOP_NUM)
top_win = np.zeros(TOP_NUM)
for a_idx,a_jg in enumerate(top_jungles):
    top_jungle_idx[a_jg] = a_idx
    
for a_idx, a_match in enumerate(matches_collection.find({'matchVersion':'6.24.168.1268'})):
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue #do not count draw game, if any

    a_duration = a_match['matchDuration']
    a_minute = a_duration/60
    if a_minute < 20:
        continue #do not count remake game
        
    if a_idx % 1000 == 0:
        print a_idx
        
    participants = a_match['participants']
    blue_ids = [p['championId'] for p in participants[0:5] if p['timeline']['lane'] == 'JUNGLE']
    red_ids = [p['championId'] for p in participants[5:10] if p['timeline']['lane'] == 'JUNGLE']
    
    #print MyCommon.JSONEncoder().encode(a_match)
    #print a_match['matchId']
    #print a_match['participantIdentities'][0]
    #print blue_ids
    #print red_ids
    for a_blue in blue_ids:
        if a_blue in top_jungles:
            if winning_side == 0:
                #blue win
                top_win[top_jungle_idx[a_blue]] += 1
            top_sum[top_jungle_idx[a_blue]] += 1
    for a_red in red_ids:
        if a_red in top_jungles:
            if winning_side == 1:
                #red win
                top_win[top_jungle_idx[a_red]] += 1
            top_sum[top_jungle_idx[a_red]] += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000


In [8]:
top_rate = top_win / top_sum
sorted_rate_idx = np.argsort(top_rate)

In [9]:
for a_idx in sorted_rate_idx:
    print top_rate[a_idx]
for a_idx in sorted_rate_idx:    
    print GetNameFromId(top_jungles[a_idx])

0.434210526316
0.458015267176
0.475903614458
0.476744186047
0.476916181085
0.478571428571
0.486092265943
0.486725663717
0.493464052288
0.496875
0.497331910352
0.503082614057
0.504201680672
0.507722007722
0.508403361345
0.514792899408
0.531052631579
0.533950617284
0.537608867775
0.561594202899
Nidalee
JarvanIV
Hecarim
Zac
Graves
Gragas
Elise
MasterYi
Olaf
Ivern
Evelynn
Rengar
Rammus
LeeSin
Vi
Kindred
Khazix
Shyvana
RekSai
Shaco


In [25]:
#KDA for 15 minutes
kill_sum = np.zeros(TOP_NUM)
assist_sum = np.zeros(TOP_NUM)
death_sum = np.zeros(TOP_NUM)
fb_sum = np.zeros(TOP_NUM)

jungle_sum = np.zeros(TOP_NUM)
cs_sum = np.zeros(TOP_NUM)
gold_sum = np.zeros(TOP_NUM)
tower_sum = np.zeros(TOP_NUM)
tower_fb_sum = np.zeros(TOP_NUM)
carry_kill_sum = np.zeros(TOP_NUM)
kda_num = np.zeros(TOP_NUM)

per_game_stats = {}
per_game_stats['blue_jg_kill'] = 0
per_game_stats['blue_jg_assist'] = 0
per_game_stats['blue_jg_death'] = 0
per_game_stats['blue_fb'] = 0
per_game_stats['blue_jg_tower'] = 0
per_game_stats['blue_fb_tower'] = 0
per_game_stats['blue_carry_kill'] = 0
per_game_stats['red_jg_kill'] = 0
per_game_stats['red_jg_assist'] = 0
per_game_stats['red_jg_death'] = 0
per_game_stats['red_fb'] = 0
per_game_stats['red_jg_tower'] = 0
per_game_stats['red_fb_tower'] = 0
per_game_stats['red_carry_kill'] = 0

CUT_FRAME = 30

def RecordBuildingKill(building_event, blue_id, red_id, participantIds, isFBTower):
    killer = participantIds[building_event['killerId'] - 1]
    assists = [participantIds[assis - 1] for assis in building_event['assistingParticipantIds']]
    
    per_game_stats['blue_jg_tower'] += 1
    if isFBTower == True:
        per_game_stats['blue_fb_tower'] = 1
            
    per_game_stats['red_jg_tower'] += 1
    if isFBTower == True:
        per_game_stats['red_fb_tower'] = 1  
        
    if 1 == 0:
        #jg kill tower
        if blue_id == killer:
            per_game_stats['blue_jg_tower'] += 1
            if isFBTower == True:
                per_game_stats['blue_fb_tower'] = 1
        if red_id == killer:
            per_game_stats['red_jg_tower'] += 1
            if isFBTower == True:
                per_game_stats['red_fb_tower'] = 1
        #jg assist tower
        if blue_id in assists:
            per_game_stats['blue_jg_tower'] += 1
            if isFBTower == True:
                per_game_stats['blue_fb_tower'] = 1
        if red_id in assists:
            per_game_stats['red_jg_tower'] += 1
            if isFBTower == True:
                per_game_stats['red_fb_tower'] = 1

def RecordKDA(kill_event, blue_id, red_id, participantIds, isFB, blue_carries, red_carries):
    #get kill, assist, death from event
    killer = participantIds[kill_event['killerId'] - 1]
    victim = participantIds[kill_event['victimId'] - 1]
    assists = [participantIds[assis - 1] for assis in kill_event['assistingParticipantIds']]
    
    isCarryKill = False
    if victim in blue_carries or victim in red_carries:
        isCarryKill = True
    
    #jg kill
    if blue_id == killer:
        per_game_stats['blue_jg_kill'] += 1
        if isFB == True:
            per_game_stats['blue_fb'] = 1
        if isCarryKill == True:
            per_game_stats['blue_carry_kill'] += 1
    if red_id == killer:
        per_game_stats['red_jg_kill'] += 1
        if isFB == True:
            per_game_stats['red_fb'] = 1
        if isCarryKill == True:
            per_game_stats['red_carry_kill'] += 1
        
    #jg death
    if blue_id == victim:
        per_game_stats['blue_jg_death'] += 1
    if red_id == victim:
        per_game_stats['red_jg_death'] += 1
        
    #jg assist
    if blue_id in assists:
        per_game_stats['blue_jg_assist'] += 1
        if isCarryKill == True:
            per_game_stats['blue_carry_kill'] += 1
    if red_id in assists:
        per_game_stats['red_jg_assist'] += 1
        if isCarryKill == True:
            per_game_stats['red_carry_kill'] += 1

for a_idx, a_match in enumerate(matches_collection.find({'matchVersion':'6.24.168.1268'})):
    winning_side = -1
    if a_match['teams'][0]['winner'] == True and a_match['teams'][1]['winner'] == False:
        winning_side = 0
    if a_match['teams'][0]['winner'] == False and a_match['teams'][1]['winner'] == True:
        winning_side = 1 
    if winning_side == -1:
        continue #do not count draw game, if any

    a_duration = a_match['matchDuration']
    a_minute = a_duration/60
    if a_minute < 20:
        continue #do not count remake game
        
    if a_idx % 1000 == 0:
        print a_idx
    
    participants = a_match['participants']
    participantIds = [p['championId'] for p in participants]
    
    blue_ids = [p['championId'] for p in participants[0:5] if p['timeline']['lane'] == 'JUNGLE']
    red_ids = [p['championId'] for p in participants[5:10] if p['timeline']['lane'] == 'JUNGLE']
    
    blue_carries = [p['championId'] for p in participants[0:5] if p['timeline']['lane'] == 'MIDDLE' or (p['timeline']['lane'] == 'BOTTOM' and p['timeline']['lane'] == 'DUO_CARRY')]
    red_carries = [p['championId'] for p in participants[5:10] if p['timeline']['lane'] == 'MIDDLE' or (p['timeline']['lane'] == 'BOTTOM' and p['timeline']['lane'] == 'DUO_CARRY')]
    
    blue_id = -1
    red_id = -1
    for a_blue in blue_ids:
        if a_blue in top_jungles:
            blue_id = a_blue
    for a_red in red_ids:
        if a_red in top_jungles:
            red_id = a_red
    
    per_game_stats['blue_jg_kill'] = 0
    per_game_stats['blue_jg_assist'] = 0
    per_game_stats['blue_jg_death'] = 0
    per_game_stats['blue_fb'] = 0
    per_game_stats['blue_jg_tower'] = 0
    per_game_stats['blue_fb_tower'] = 0
    per_game_stats['blue_carry_kill'] = 0
    per_game_stats['red_jg_kill'] = 0
    per_game_stats['red_jg_assist'] = 0
    per_game_stats['red_jg_death'] = 0
    per_game_stats['red_fb'] = 0
    per_game_stats['red_jg_tower'] = 0
    per_game_stats['red_fb_tower'] = 0
    per_game_stats['red_carry_kill'] = 0
    isFB = True
    isFBTower = True
    timeline_frames = a_match['timeline']['frames']
    
    #if len(timeline_frames) < CUT_FRAME:
        #continue
    CUT_FRAME = len(timeline_frames)
        
    for frame_idx, each_frame in enumerate(timeline_frames):
        if frame_idx >= CUT_FRAME:
            break
        events = each_frame['events']
        for event_idx, each_event in enumerate(events):
            if each_event['eventType'] == 'CHAMPION_KILL':
                #print each_event
                RecordKDA(each_event, blue_id, red_id, participantIds, isFB, blue_carries, red_carries)
                isFB = False
            if each_event['eventType'] == 'BUILDING_KILL':
                RecordBuildingKill(each_event, blue_id, red_id, participantIds, isFBTower)
                isFBTower = False
                
    #get 15 minutes KDA for both side
    if blue_id != -1:
        kill_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_jg_kill']
        assist_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_jg_assist']
        death_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_jg_death']
        fb_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_fb']
        tower_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_jg_tower']
        tower_fb_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_fb_tower']
        carry_kill_sum[top_jungle_idx[blue_id]] += per_game_stats['blue_carry_kill']

        blue_idx = str(participantIds.index(blue_id) + 1)
        gold_sum[top_jungle_idx[blue_id]] += timeline_frames[CUT_FRAME-1]['participantFrames'][blue_idx]['totalGold']
        jungle_sum[top_jungle_idx[blue_id]] += timeline_frames[CUT_FRAME-1]['participantFrames'][blue_idx]['jungleMinionsKilled']
        cs_sum[top_jungle_idx[blue_id]] += timeline_frames[CUT_FRAME-1]['participantFrames'][blue_idx]['minionsKilled']
        kda_num[top_jungle_idx[blue_id]] += 1
    if red_id != -1:
        kill_sum[top_jungle_idx[red_id]] += per_game_stats['red_jg_kill']
        assist_sum[top_jungle_idx[red_id]] += per_game_stats['red_jg_assist']
        death_sum[top_jungle_idx[red_id]] += per_game_stats['red_jg_death']
        fb_sum[top_jungle_idx[red_id]] += per_game_stats['red_fb']
        tower_sum[top_jungle_idx[red_id]] += per_game_stats['red_jg_tower']
        tower_fb_sum[top_jungle_idx[red_id]] += per_game_stats['red_fb_tower']
        carry_kill_sum[top_jungle_idx[red_id]] += per_game_stats['red_carry_kill']
        
        red_idx = str(participantIds.index(red_id) + 1)
        gold_sum[top_jungle_idx[red_id]] += timeline_frames[CUT_FRAME-1]['participantFrames'][red_idx]['totalGold']
        jungle_sum[top_jungle_idx[red_id]] += timeline_frames[CUT_FRAME-1]['participantFrames'][red_idx]['jungleMinionsKilled']
        cs_sum[top_jungle_idx[red_id]] += timeline_frames[CUT_FRAME-1]['participantFrames'][red_idx]['minionsKilled']

        kda_num[top_jungle_idx[red_id]] += 1

    
print kill_sum
print death_sum
print assist_sum
print fb_sum
print jungle_sum
print cs_sum
print gold_sum
print kda_num

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
[   850.    435.    663.    608.    963.    694.   1474.   1904.   1898.
   1009.   1960.   4019.   6432.   5487.   7278.  16252.  14964.  12839.
  20446.  36048.]
[   713.    614.    742.    696.   1029.    731.   1376.   1425.   1536.
   1337.   1513.   3403.   4915.   4060.   5017.  10183.  12366.  10577.
  16530.  28273.]
[   543.   1337.   1488.   1378.   1189.   1666.   1693.   1877.   2495.
   3849.   2388.   4305.   5176.   6577.   8375.  11513.  14681.  24198.
  23096.  41986.]
[  15.   12.   22.   15.   22.   12.   26.   42.   40.   13.   19.  100.
  131.   95.  191.  338.  325.  316.  560.  974.]
[  10062.    8490.    9905.   12122.   14234.   14777.   21338.   21230.
   27009.   20303.   36270.   67524.   87681.   78975.   75557.  170820.
  220715.  270073.  224609.  446923.]
[   6721.    4258.    5048.    5691.    6476.    6083.    8473.   12128.
   13022.    6653.   15761.   23381.   34862.   34432.   32106.   690

In [24]:
average_kill = kill_sum / kda_num
average_assist = assist_sum / kda_num
average_death = death_sum / kda_num
average_fb = fb_sum / kda_num
average_jungle = jungle_sum / kda_num
average_cs = cs_sum / kda_num
average_gold = gold_sum / kda_num
average_tower = tower_sum / kda_num
average_fb_tower = tower_fb_sum / kda_num
average_carry_kill = carry_kill_sum / kda_num
print average_kill
print average_assist
print average_death
print average_fb
print average_jungle
print average_cs
print average_gold
print average_tower
print average_fb_tower
print average_carry_kill

[ 7.58928571  3.65546218  5.0610687   4.34285714  5.69822485  4.05847953
  6.19327731  6.89855072  6.22295082  3.1630094   6.04938272  6.62108731
  7.95055624  6.61084337  7.76734258  8.55818852  6.71633752  5.08475248
  6.93790295  6.96983759]
[  4.84821429  11.23529412  11.35877863   9.84285714   7.03550296
   9.74269006   7.11344538   6.80072464   8.18032787  12.06583072
   7.37037037   7.092257     6.39802225   7.92409639   8.93810032
   6.06266456   6.58931777   9.58336634   7.8371225    8.11794277]
[ 6.36607143  5.15966387  5.66412214  4.97142857  6.0887574   4.2748538
  5.78151261  5.16304348  5.03606557  4.19122257  4.66975309  5.6062603
  6.07540173  4.89156627  5.35432231  5.36229595  5.5502693   4.18891089
  5.60909399  5.46655066]
[ 0.13392857  0.10084034  0.16793893  0.10714286  0.13017751  0.07017544
  0.1092437   0.15217391  0.13114754  0.04075235  0.05864198  0.16474465
  0.16192831  0.11445783  0.20384205  0.17798841  0.14587074  0.12514851
  0.19002375  0.18832173]
[ 

In [13]:
average_kda = (average_kill + average_assist) / average_death
print average_kda
for jg in top_jungles:
    print GetNameFromId(jg)

[ 2.04010695  3.00684932  2.82555283  2.66847826  1.93453355  3.01002506
  2.3033557   2.96948682  2.78359909  3.05216285  2.75966184  2.38100209
  2.49479941  2.82303246  3.19759825  2.89235075  2.42459605  3.29667221
  2.79722589  2.73551872]
MasterYi
Rammus
JarvanIV
Gragas
Kindred
Zac
Vi
Shaco
Olaf
Ivern
Shyvana
Nidalee
Rengar
Hecarim
Evelynn
Khazix
Graves
RekSai
Elise
LeeSin


In [76]:
for value in top_rate:
    print value


0.486725663717
0.504201680672
0.458015267176
0.478571428571
0.514792899408
0.476744186047
0.508403361345
0.561594202899
0.493464052288
0.496875
0.533950617284
0.434210526316
0.503082614057
0.475903614458
0.497331910352
0.531052631579
0.476916181085
0.537608867775
0.486092265943
0.507722007722


In [58]:
print participantIds

[13, 60, 245, 89, 81, 15, 43, 104, 4, 58]


1